In [1]:
import pandas as pd
from pandas import json_normalize
import numpy as np
import json as js
import ast as ast

In [ ]:
list_reviews = []
archivo1 = r'australian_user_reviews.json'
with open(archivo1, encoding='utf-8') as file:
    for line in file.readlines():
        list_reviews.append(ast.literal_eval(line))
list_reviews

In [ ]:
reviews_se = pd.DataFrame(list_reviews)
reviews = reviews_se.explode('reviews', ignore_index=True)
reviews

In [ ]:
reviews_expanded = pd.json_normalize(reviews['reviews'])
reviews_expanded

In [ ]:
reviews_expanded=reviews_expanded.replace('',None)
reviews_expanded

In [ ]:
reviews_def = reviews.join(reviews_expanded)
reviews_def

In [ ]:
reviews_def.drop(columns=['reviews'],inplace=True)
reviews_def

In [ ]:
reviews_def = reviews_def.drop(columns=['user_url', 'funny', 'last_edited', 'helpful'])

In [ ]:
reviews = reviews.drop_duplicates()